In [119]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import json
import joblib

with open("stocks.json") as json_file:
    stock_paths = json.load(json_file)
    predict_path = stock_paths["lf_predict"]
    other_paths = stock_paths["lf"]
    other_paths.append(predict_path)

In [120]:
stocks = [pd.read_csv(f"../data/{path}") for path in other_paths]

In [121]:
def prepare_stocks(df, start_date, end_date):
    df = df.drop(columns=["status"])
    # df["per_change"] = df["per_change"].fillna(0)
    df["published_date"] = pd.to_datetime(df["published_date"])
    df.set_index("published_date", inplace=True)
    df = df[df.index >= start_date]
    if df.isnull().values.any() or np.isinf(df.values).any():
        df = df.fillna(df.mean())
        df = df.replace([np.inf, -np.inf], np.nan)
        df = df.fillna(df.mean())
    df = df[df.index < end_date]
    return df

In [ ]:
for i, path in enumerate(other_paths):
    print(i, " ", path, " ", stocks[i].shape)

In [ ]:
stocks = [prepare_stocks(stock, "2011-03-29", "2022-12-22") for stock in stocks]
stocks = pd.concat(stocks)
stocks.shape

In [ ]:
scaler = StandardScaler()
scaler.fit(stocks)

joblib.dump(scaler, "./scalers/lf.pkl")